In [2]:
import xml.etree.ElementTree as ET

# Path to the OSM file


# Parse the OSM file
tree = ET.parse(r"C:\Users\FPTSHOP\Downloads\map (2).osm")
root = tree.getroot()

# Extract nodes and their coordinates
nodes = {}
for element in root:
    if element.tag == 'node':
        node_id = element.attrib['id']
        lat = float(element.attrib['lat'])
        lon = float(element.attrib['lon'])
        nodes[node_id] = (lat, lon)

# Display the number of nodes found and a few examples
num_nodes = len(nodes)
example_nodes = list(nodes.items())[:5]

num_nodes, example_nodes



(51984,
 [('74126839', (21.0314522, 105.8529442)),
  ('74126842', (21.0316279, 105.8520348)),
  ('74126845', (21.0316478, 105.8516382)),
  ('74126847', (21.0315876, 105.8514927)),
  ('74126849', (21.0315049, 105.8514137))])

In [3]:
from collections import defaultdict

# Function to find adjacencies
def find_adjacencies(root):
    adjacencies = defaultdict(set)
    for element in root:
        if element.tag == 'way':
            # Extract the node IDs in this way
            way_nodes = [nd.attrib['ref'] for nd in element if nd.tag == 'nd']
            # Add adjacencies
            for i in range(len(way_nodes) - 1):
                adjacencies[way_nodes[i]].add(way_nodes[i + 1])
                adjacencies[way_nodes[i + 1]].add(way_nodes[i])
    return adjacencies

# Find adjacencies between nodes
adjacencies = find_adjacencies(root)

# Display the number of adjacencies and a few examples
num_adjacencies = len(adjacencies)
example_adjacencies = {k: v for k, v in list(adjacencies.items())[:5]}

num_adjacencies, example_adjacencies



(48424,
 {'309641646': {'11304581471'},
  '11304581471': {'11304264839', '11304581472', '309641646'},
  '11304264839': {'11304581471', '2169312366'},
  '2169312366': {'11304264839', '2169312365'},
  '2169312365': {'11304264840', '2169312366'}})

In [4]:
coord_to_index = {coord: int(idx) for idx, coord in nodes.items()}
index_to_coord = {int(idx): coord for idx, coord in nodes.items()}

In [5]:
import numpy as np
import scipy.sparse as sparse

# Create a mapping of node ID to a unique index
node_to_index = {node_id: idx for idx, node_id in enumerate(nodes.keys())}

# Prepare data for the sparse matrix
rows = []
cols = []

for node_id, adjacent_nodes in adjacencies.items():
    node_idx = node_to_index[node_id]
    for adj_node_id in adjacent_nodes:
        adj_node_idx = node_to_index[adj_node_id]
        rows.append(node_idx)
        cols.append(adj_node_idx)

# Create the sparse matrix in COO format
data = np.ones(len(rows))  # Use 1 to indicate an edge
adjacency_matrix = sparse.coo_matrix((data, (rows, cols)), shape=(num_nodes, num_nodes))

# Display the shape of the matrix and a small part of it as an example
matrix_shape = adjacency_matrix.shape






In [6]:
from queue import PriorityQueue
import numpy as np

def haversine(coord1, coord2):
    # Calculate the Haversine distance between two coordinates in (lon, lat) format
    lon1, lat1 = coord1
    lon2, lat2 = coord2
    R = 6371  # Radius of the Earth in kilometers
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

def a_star(start_coord, goal_coord, adjacencies, coord_to_index, index_to_coord):
    start = coord_to_index[start_coord]
    goal = coord_to_index[goal_coord]

    open_set = PriorityQueue()
    open_set.put((0, start))

    came_from = {start: None}
    cost_so_far = {start: 0}

    node_seen_count = {}
    explored_routes = []

    while not open_set.empty():
        current = open_set.get()[1]
        node_seen_count[current] = node_seen_count.get(current, 0) + 1
        
        if current == goal:
            break

        current_coord = index_to_coord[int(current)]

        # Access only neighbors of the current node
        for next_node in adjacencies.get(str(current), []):
            next_node = int(next_node)
            next_node_coord = index_to_coord[int(next_node)]

            # Calculate distance using Haversine formula
            distance = haversine(current_coord, next_node_coord)
            new_cost = cost_so_far[current] + distance

            if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                cost_so_far[next_node] = new_cost
                priority = new_cost + haversine(next_node_coord, goal_coord)
                open_set.put((priority, next_node))
                came_from[next_node] = current
                explored_routes.append((current, next_node))

    # Reconstruct path
    path = []
    current = goal
    while current != start:
        if current not in came_from:
            break
        path.append(index_to_coord[current])
        current = came_from[current]
    path.append(index_to_coord[start])
    path.reverse()

    return path, cost_so_far.get(goal, float('inf')), node_seen_count, explored_routes


In [7]:
def UCS(start_coord, goal_coord, adjacencies, coord_to_index, index_to_coord):
    start = coord_to_index[start_coord]
    goal = coord_to_index[goal_coord]

    open_set = PriorityQueue()
    open_set.put((0, start))

    came_from = {start: None}
    cost_so_far = {start: 0}

    node_seen_count = {}
    explored_routes = []

    while not open_set.empty():
        current = open_set.get()[1]
        node_seen_count[current] = node_seen_count.get(current, 0) + 1
        
        if current == goal:
            break

        current_coord = index_to_coord[int(current)]

        # Access only neighbors of the current node
        for next_node in adjacencies.get(str(current), []):
            next_node = int(next_node)
            next_node_coord = index_to_coord[int(next_node)]

            # Calculate distance using Haversine formula
            distance = haversine(current_coord, next_node_coord)
            new_cost = cost_so_far[current] + distance

            if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                cost_so_far[next_node] = new_cost
                priority = new_cost 
                open_set.put((priority, next_node))
                came_from[next_node] = current
                explored_routes.append((current, next_node))

    # Reconstruct path
    path = []
    current = goal
    while current != start:
        if current not in came_from:
            break
        path.append(index_to_coord[current])
        current = came_from[current]
    path.append(index_to_coord[start])
    path.reverse()

    return path, cost_so_far.get(goal, float('inf')), node_seen_count, explored_routes


In [8]:
def a_star_plus(start_coord, goal_coord, adjacencies, coord_to_index, index_to_coord):
    start = coord_to_index[start_coord]
    goal = coord_to_index[goal_coord]
    air_distance = haversine(start_coord,goal_coord)
    open_set = PriorityQueue()
    open_set.put((0, start))

    came_from = {start: None}
    cost_so_far = {start: 0}

    node_seen_count = {}
    explored_routes = []

    while not open_set.empty():
        current = open_set.get()[1]
        node_seen_count[current] = node_seen_count.get(current, 0) + 1
        
        if current == goal:
            break

        current_coord = index_to_coord[int(current)]

        # Access only neighbors of the current node
        for next_node in adjacencies.get(str(current), []):
            next_node = int(next_node)
            next_node_coord = index_to_coord[int(next_node)]

            # Calculate distance using Haversine formula
            distance = haversine(current_coord, next_node_coord)
            new_cost = cost_so_far[current] + distance

            if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                cost_so_far[next_node] = new_cost
                heuristic = haversine(next_node_coord, goal_coord)
                if heuristic>=air_distance*0.7:
                    priority = new_cost + heuristic
                else :
                    priority = new_cost + heuristic*0.65
                open_set.put((priority, next_node))
                came_from[next_node] = current
                explored_routes.append((current, next_node))

    # Reconstruct path
    path = []
    current = goal
    while current != start:
        if current not in came_from:
            break
        path.append(index_to_coord[current])
        current = came_from[current]
    path.append(index_to_coord[start])
    path.reverse()

    return path, cost_so_far.get(goal, float('inf')), node_seen_count, explored_routes


In [14]:
def prune_open_set(open_set, memory_limit):
    if open_set.qsize() > memory_limit:
        # Keep the nodes with the lowest estimated total cost
        kept_nodes = [open_set.get() for _ in range(memory_limit)]
        open_set.queue.clear()
        for item in kept_nodes:
            open_set.put(item)

def a_star_memory_bounded(start_coord, goal_coord, adjacencies, coord_to_index, index_to_coord, memory_limit):
    start = coord_to_index[start_coord]
    goal = coord_to_index[goal_coord]
    air_distance = haversine(start_coord,goal_coord)
    open_set = PriorityQueue()
    open_set.put((0, start))

    came_from = {start: None}
    cost_so_far = {start: 0}

    node_seen_count = {}
    explored_routes = []

    while not open_set.empty():
        prune_open_set(open_set, memory_limit)  # Prune the open set to meet memory constraints

        current = open_set.get()[1]
        node_seen_count[current] = node_seen_count.get(current, 0) + 1
        
        if current == goal:
            break

        current_coord = index_to_coord[int(current)]

        # Access only neighbors of the current node
        for next_node in adjacencies.get(str(current), []):
            next_node = int(next_node)
            next_node_coord = index_to_coord[int(next_node)]

            # Calculate distance using Haversine formula
            distance = haversine(current_coord, next_node_coord)
            new_cost = cost_so_far[current] + distance

            if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                cost_so_far[next_node] = new_cost
                heuristic = haversine(next_node_coord, goal_coord)
                if heuristic>=air_distance*0.7:
                    priority = new_cost + heuristic
                else :
                    priority = new_cost + heuristic*0.65
                open_set.put((priority, next_node))
                came_from[next_node] = current
                explored_routes.append((current, next_node))

    # Reconstruct path
    path = []
    current = goal
    while current != start:
        if current not in came_from:
            break
        path.append(index_to_coord[current])
        current = came_from[current]
    path.append(index_to_coord[start])
    path.reverse()

    return path, cost_so_far.get(goal, float('inf')), node_seen_count, explored_routes



In [10]:
coord_to_index = {coord: int(idx) for idx, coord in nodes.items()}
index_to_coord = {int(idx): coord for idx, coord in nodes.items()}

In [17]:
memory_limit =100
path, cost, node_seen_count, explored_routes=UCS(start_coord=(21.0314522,105.8529442,),goal_coord=( 21.0072383, 105.8348649),adjacencies=adjacencies,coord_to_index=coord_to_index,index_to_coord=index_to_coord)
path, cost, node_seen_count, explored_routes=a_star(start_coord=(21.0314522,105.8529442,),goal_coord=( 21.0072383, 105.8348649),adjacencies=adjacencies,coord_to_index=coord_to_index,index_to_coord=index_to_coord)
path, cost, node_seen_count, explored_routes=a_star_plus(start_coord=(21.0314522,105.8529442,),goal_coord=( 21.0072383, 105.8348649),adjacencies=adjacencies,coord_to_index=coord_to_index,index_to_coord=index_to_coord)
path, cost, node_seen_count, explored_routes=a_star_memory_bounded(start_coord=(21.0314522,105.8529442,),goal_coord=( 21.0072383, 105.8348649),adjacencies=adjacencies,coord_to_index=coord_to_index,index_to_coord=index_to_coord,memory_limit=memory_limit)

In [19]:
import folium

def plot_combined_searches(paths, costs, node_seen_counts, explored_routes_list, index_to_coord):
    if any(not path for path in paths):
        raise ValueError("One or more paths are empty, cannot visualize.")

    # Initialize the map centered around the start of the first path
    m = folium.Map(location=paths[0][0], zoom_start=14)

    colors = ['blue', 'green', 'purple', 'orange']  # Different colors for each path
    for i, path in enumerate(paths):
        # Add markers for the start and end of each path
        folium.Marker(path[0], popup=f'Start of path {i+1}', icon=folium.Icon(color=colors[i])).add_to(m)
        folium.Marker(path[-1], popup=f'Goal of path {i+1}', icon=folium.Icon(color='red')).add_to(m)

        # Add the path
        folium.PolyLine(path, color=colors[i], weight=2.5, opacity=1, popup=f'Path {i+1} Cost: {costs[i]}').add_to(m)

        # Add markers for nodes seen
        for node, count in node_seen_counts[i].items():
            if node in index_to_coord:
                coord = index_to_coord[node]
                folium.CircleMarker(coord, radius=5, color=colors[i], fill=True, fill_color=colors[i], popup=f'Path {i+1} Seen {count} times').add_to(m)

        # Add lines for explored routes
        for route in explored_routes_list[i]:
            start_node, end_node = route
            if start_node in index_to_coord and end_node in index_to_coord:
                start_coord = index_to_coord[start_node]
                end_coord = index_to_coord[end_node]
                folium.PolyLine([start_coord, end_coord], color=colors[i], weight=1, opacity=0.5).add_to(m)

    return m





In [21]:
import folium


# Run the search algorithms and store their outputs
memory_limit = 100
path_UCS, cost_UCS, node_seen_count_UCS, explored_routes_UCS = UCS(start_coord=(21.0314522,105.8529442), goal_coord=(21.0072383, 105.8348649), adjacencies=adjacencies, coord_to_index=coord_to_index, index_to_coord=index_to_coord)
path_A_star, cost_A_star, node_seen_count_A_star, explored_routes_A_star = a_star(start_coord=(21.0314522,105.8529442), goal_coord=(21.0072383, 105.8348649), adjacencies=adjacencies, coord_to_index=coord_to_index, index_to_coord=index_to_coord)
path_A_star_plus, cost_A_star_plus, node_seen_count_A_star_plus, explored_routes_A_star_plus = a_star_plus(start_coord=(21.0314522,105.8529442), goal_coord=(21.0072383, 105.8348649), adjacencies=adjacencies, coord_to_index=coord_to_index, index_to_coord=index_to_coord)
path_A_star_memory_bounded, cost_A_star_memory_bounded, node_seen_count_A_star_memory_bounded, explored_routes_A_star_memory_bounded = a_star_memory_bounded(start_coord=(21.0314522,105.8529442), goal_coord=(21.0072383, 105.8348649), adjacencies=adjacencies, coord_to_index=coord_to_index, index_to_coord=index_to_coord, memory_limit=memory_limit)

# Prepare data for visualization
paths = [path_UCS, path_A_star, path_A_star_plus, path_A_star_memory_bounded]
costs = [cost_UCS, cost_A_star, cost_A_star_plus, cost_A_star_memory_bounded]
node_seen_counts = [node_seen_count_UCS, node_seen_count_A_star, node_seen_count_A_star_plus, node_seen_count_A_star_memory_bounded]
explored_routes_list = [explored_routes_UCS, explored_routes_A_star, explored_routes_A_star_plus, explored_routes_A_star_memory_bounded]

# Generate and save the map visualization
map_visualization = plot_combined_searches(paths, costs, node_seen_counts, explored_routes_list, index_to_coord)
map_visualization.save('combined_search_visualization.html')
